In [1]:
import os, sys
sys.path.append('..')
import common, sql
import pandas as pd

In [2]:
q='''select * from tool
where id not in (1);'''
tools=sql.execute(q)
tools

[{'id': 2, 'name': 'Snyk'},
 {'id': 3, 'name': 'Github Dependabot'},
 {'id': 4, 'name': 'OWASP Dependency-Check'},
 {'id': 5, 'name': 'NPM Audit'},
 {'id': 6, 'name': 'Contrast'},
 {'id': 7, 'name': 'WhiteSource'},
 {'id': 8, 'name': 'Seeker'},
 {'id': 9, 'name': 'Steady'}]

In [3]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.alerts,0) as alerts from repository r
        left join
        (select repositoryId, sum(nA.count)as alerts from npmAlert nA
        join dependency d on nA.dependencyId = d.id
        join vulnerability v on nA.vulnerabilityId = v.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id in (8,20,25,33,44)
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.alerts.sum(), df.alerts.median())

Snyk 2210 135.0
Github Dependabot 97 8.0
OWASP Dependency-Check 1379 208.0
NPM Audit 1266 37.0
Contrast 0 0.0
WhiteSource 205 32.0
Seeker 0 0.0
Steady 0 0.0


In [4]:
q='''select t.id, t.name, count(distinct nA.dependencyPathId)
from npmAlert nA
join dependency d on nA.dependencyId = d.id
join package p on d.packageId = p.id
join vulnerability v on nA.vulnerabilityId = v.id
join tool t on nA.toolId = t.id
group by t.id, t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct nA.dependencyPathId)
0,2,Snyk,1004
1,3,Github Dependabot,0
2,4,OWASP Dependency-Check,498
3,5,NPM Audit,852
4,7,WhiteSource,0


In [5]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.c,0) as c from repository r
        left join
        (select repositoryId, count(distinct nA.dependencyPathId) as c
        from npmAlert nA
        join dependency d on nA.dependencyId = d.id
        join package p on d.packageId = p.id
        join vulnerability v on nA.vulnerabilityId = v.id
        join tool t on nA.toolId = t.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id in (8,20,25,33,44)
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.c.median())

Snyk 44.0
Github Dependabot 0.0
OWASP Dependency-Check 72.0
NPM Audit 28.0
Contrast 0.0
WhiteSource 0.0
Seeker 0.0
Steady 0.0


In [6]:
q='''select t.id, t.name, count(distinct d.packageId)
from npmAlert nA
join dependency d on nA.dependencyId = d.id
join package p on d.packageId = p.id
join vulnerability v on nA.vulnerabilityId = v.id
join tool t on nA.toolId = t.id
group by t.id, t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct d.packageId)
0,2,Snyk,90
1,3,Github Dependabot,32
2,4,OWASP Dependency-Check,239
3,5,NPM Audit,58
4,7,WhiteSource,89


In [7]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.c,0) as c from repository r
        left join
        (select repositoryId, count(distinct d.packageId) as c
        from npmAlert nA
        join dependency d on nA.dependencyId = d.id
        join package p on d.packageId = p.id
        join vulnerability v on nA.vulnerabilityId = v.id
        join tool t on nA.toolId = t.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id in (8,20,25,33,44)
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.c.median())

Snyk 20.0
Github Dependabot 1.0
OWASP Dependency-Check 71.0
NPM Audit 12.0
Contrast 0.0
WhiteSource 14.0
Seeker 0.0
Steady 0.0


In [8]:
q='''select t.id, t.name, count(distinct p.artifact)
from npmAlert nA
join dependency d on nA.dependencyId = d.id
join package p on d.packageId = p.id
join vulnerability v on nA.vulnerabilityId = v.id
join tool t on nA.toolId = t.id
group by t.id, t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct p.artifact)
0,2,Snyk,54
1,3,Github Dependabot,30
2,4,OWASP Dependency-Check,160
3,5,NPM Audit,45
4,7,WhiteSource,55


In [9]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.c,0) as c from repository r
        left join
        (select repositoryId, count(distinct p.artifact) as c
        from npmAlert nA
        join dependency d on nA.dependencyId = d.id
        join package p on d.packageId = p.id
        join vulnerability v on nA.vulnerabilityId = v.id
        join tool t on nA.toolId = t.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id in (8,20,25,33,44)
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.c.median())

Snyk 17.0
Github Dependabot 1.0
OWASP Dependency-Check 57.0
NPM Audit 12.0
Contrast 0.0
WhiteSource 9.0
Seeker 0.0
Steady 0.0


In [10]:
q='''select t.id, t.name, count(distinct v.id)
from npmAlert nA
join dependency d on nA.dependencyId = d.id
join package p on d.packageId = p.id
join vulnerability v on nA.vulnerabilityId = v.id
join tool t on nA.toolId = t.id
group by t.id, t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct v.id)
0,2,Snyk,121
1,3,Github Dependabot,45
2,4,OWASP Dependency-Check,234
3,5,NPM Audit,62
4,7,WhiteSource,96


In [11]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.c,0) as c from repository r
        left join
        (select repositoryId, count(distinct v.id) as c
        from npmAlert nA
        join dependency d on nA.dependencyId = d.id
        join package p on d.packageId = p.id
        join vulnerability v on nA.vulnerabilityId = v.id
        join tool t on nA.toolId = t.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id in (8,20,25,33,44)
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.c.median())

Snyk 26.0
Github Dependabot 4.0
OWASP Dependency-Check 71.0
NPM Audit 16.0
Contrast 0.0
WhiteSource 18.0
Seeker 0.0
Steady 0.0


In [12]:
q='''select t.id, t.name, count(distinct v.id)
from npmAlert nA
join dependency d on nA.dependencyId = d.id
join package p on d.packageId = p.id
join vulnerability v on nA.vulnerabilityId = v.id
join tool t on nA.toolId = t.id
where v.cveId is not null and v.sourceId is null
group by t.id, t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct v.id)
0,2,Snyk,79
1,3,Github Dependabot,29
2,4,OWASP Dependency-Check,78
3,5,NPM Audit,31
4,7,WhiteSource,58


In [13]:
q='''select t.id, t.name, count(distinct v.id)
from npmAlert nA
join dependency d on nA.dependencyId = d.id
join package p on d.packageId = p.id
join vulnerability v on nA.vulnerabilityId = v.id
join tool t on nA.toolId = t.id
where v.cveId is null and v.sourceId is not null
group by t.id, t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct v.id)
0,2,Snyk,42
1,3,Github Dependabot,16
2,4,OWASP Dependency-Check,156
3,5,NPM Audit,31
4,7,WhiteSource,38


In [14]:
q='select * from scanMinutes'
sql.execute(q)

[{'toolId': 1, 'maven': 3.43333, 'npm': None},
 {'toolId': 2, 'maven': 15.1333, 'npm': 1.05},
 {'toolId': 4, 'maven': 14.4167, 'npm': 4.68333},
 {'toolId': 5, 'maven': None, 'npm': 0.133333},
 {'toolId': 9, 'maven': 385.0, 'npm': None}]